In [1]:
from dash import Dash, dcc, html, Input, Output, callback
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objs as go
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import viridis

In [2]:
df = pd.read_csv('Combined.csv')
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(df['Duration'])], [int(round((i%1) * 60, 2)) for i in list(df['Duration'])])]
df['Track Duration'] = duration_mins_secs
df.sample(5)

,Unnamed: 0,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,...,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Playlist,Track Duration
53,53,Summer of Love (Shawn Mendes & Tainy),spotify:track:0z8hI3OPS8ADPWtoCjjLl6,https://open.spotify.com/track/0z8hI3OPS8ADPWt...,"['Shawn Mendes', 'Tainy']",Summer Of Love,75,3.07,123.988,11,...,0.808,-4.501,0.1170,0.0297,0.000127,0.1030,0.494,4,Playlist 2,3:4
52,52,Light Switch,spotify:track:1jEBSDN5vYViJQr78W7jr2,https://open.spotify.com/track/1jEBSDN5vYViJQr...,['Charlie Puth'],CHARLIE,78,3.13,184.008,6,...,0.627,-5.692,0.3080,0.1050,0.000321,0.0905,0.905,4,Playlist 2,3:7
69,69,Counting Stars,spotify:track:2tpWsVSb9UEmDRxAl1zhX1,https://open.spotify.com/track/2tpWsVSb9UEmDRx...,['OneRepublic'],Native,88,4.29,122.016,1,...,0.705,-4.972,0.0382,0.0654,0.000000,0.1180,0.477,4,Playlist 2,4:17
36,36,Yesterday - Remastered 2009,spotify:track:3BQHpFgAp4l80e1XslIjNI,https://open.spotify.com/track/3BQHpFgAp4l80e1...,['The Beatles'],Help! (Remastered),80,2.09,96.529,5,...,0.179,-11.830,0.0326,0.8790,0.000000,0.0886,0.315,4,Playlist 1,2:5
76,76,Wildest Dreams,spotify:track:3fVnlF4pGqWI9flVENcT28,https://open.spotify.com/track/3fVnlF4pGqWI9fl...,['Taylor Swift'],1989,81,3.67,140.060,8,...,0.664,-7.417,0.0741,0.0709,0.005600,0.1060,0.467,4,Playlist 2,3:40


In [3]:
p1 = pd.read_csv('Playlist1.csv')
p1 = pd.DataFrame(p1[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p2 = pd.read_csv('Playlist2.csv')
p2 = pd.DataFrame(p2[['Acousticness', 'Danceability', 'Energy', 'Instrumentalness', 'Speechiness', 'Valence']].mean()).T
p = pd.concat([p1, p2], ignore_index=True)
p

,Acousticness,Danceability,Energy,Instrumentalness,Speechiness,Valence
0,0.323171,0.49882,0.54488,0.051619,0.035736,0.346190
1,0.134577,0.64820,0.71412,0.012604,0.061402,0.557222


In [4]:
artists = pd.read_csv('All Artists.csv')
artists['Index'] = artists.index
artists.sample(5)

,Name,ID,Genres,Followers,Popularity,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
9,Oasis,2DaxqgrOhkeH0fpeiQq2f4,"['beatlesque', 'britpop', 'madchester', 'perma...",9425815,74,Playlist 1,2,2,0,9
3,George Ezra,2ysnwxxNtSgbb9t1m2Ur4j,"['folk-pop', 'neo-singer-songwriter', 'talent ...",4146533,71,Playlist 1,1,1,0,3
69,Bruno Mars,0du5cEVh5yTK9QJze8zA0C,"['dance pop', 'pop']",52578040,86,Playlist 2,2,0,2,69
57,Justin Bieber,1uNFoZAHBGtllmzznpCI3s,"['canadian pop', 'pop']",74884572,87,Playlist 2,2,0,2,57
13,Hozier,2FXC3k01G6Gw61bmprjgqS,"['irish singer-songwriter', 'modern rock', 'po...",6183294,80,Playlist 1,1,1,0,13


In [5]:
genres = pd.read_csv('Genres.csv')
genres['Index'] = genres.index
genres.sample(5)

,Genre,Playlist,Total Count,Count in Playlist 1,Count in Playlist 2,Index
17,viral pop,Playlist 2,2,0,2,17
10,trap queen,Playlist 2,2,0,2,10
37,indie rock,Playlist 2,1,0,1,37
42,minnesota hip hop,Playlist 2,1,0,1,42
113,alternative metal,Playlist 1,1,1,0,113


In [6]:
collabs = pd.read_csv('All Collaborations.csv')
collabs.sample(5)

,Artist 1,Artist 2,Popular Collab,URL,Popularity,Artist From
22,Coldplay,BTS,My Universe,https://open.spotify.com/track/3FeVmId7tL5YN8B...,81.0,Artists in Different Playlists
105,Ariana Grande,Miley Cyrus,Don’t Call Me Angel (Charlie’s Angels) (with M...,https://open.spotify.com/track/6zegtH6XXd2PDPL...,71.0,Artists from Same Playlist
99,Selena Gomez,Marshmello,Wolves,https://open.spotify.com/track/0tBbt8CrmxbjRP0...,80.0,Artists from Same Playlist
134,Lizzo,Ariana Grande,Good as Hell (feat. Ariana Grande),https://open.spotify.com/track/2rjjNUYONZqC7wS...,62.0,Artists from Same Playlist
12,Ed Sheeran,Bruno Mars,BLOW (with Chris Stapleton & Bruno Mars),https://open.spotify.com/track/6EbduYTr62sSzvl...,64.0,Artists in Different Playlists


In [7]:
artists_songs = pd.read_csv("Common Artist Songs.csv")
artists_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Artist
20,Maps,spotify:track:4gbVRS8gloEluzf0GzDOFc,https://open.spotify.com/track/4gbVRS8gloEluzf...,['Maroon 5'],V,86,3.17,120.032,1,0,0.742,0.713,-5.522,0.0303,0.02050,0.000,0.0590,0.879,4,Maroon 5
22,This Love,spotify:track:6ECp64rv50XVz93WvxXMGF,https://open.spotify.com/track/6ECp64rv50XVz93...,['Maroon 5'],Songs About Jane: 10th Anniversary Edition,84,3.44,95.051,5,0,0.712,0.862,-4.612,0.0378,0.05250,0.000,0.0930,0.809,4,Maroon 5
5,House of Memories,spotify:track:2DgdHcjWmO3qd50RzuBLgZ,https://open.spotify.com/track/2DgdHcjWmO3qd50...,['Panic! At The Disco'],Death of a Bachelor,83,3.48,109.952,11,0,0.506,0.817,-5.051,0.0335,0.00272,0.000,0.0486,0.506,4,Panic! At The Disco
33,Lovesong - 2010 Remaster,spotify:track:0X5C4WjQNubRysTkHOubz3,https://open.spotify.com/track/0X5C4WjQNubRysT...,['The Cure'],Disintegration (Deluxe Edition),67,3.49,140.112,9,0,0.611,0.813,-7.689,0.0279,0.33900,0.898,0.2240,0.821,4,The Cure
21,Memories,spotify:track:4cktbXiXOapiLBMprHFErI,https://open.spotify.com/track/4cktbXiXOapiLBM...,['Maroon 5'],JORDI (Deluxe),86,3.16,91.050,11,1,0.775,0.327,-7.241,0.0557,0.84100,0.000,0.0821,0.595,4,Maroon 5


In [8]:
genre_songs = pd.read_csv("Common Genre Songs.csv")
genre_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Common Genre
118,Is It Any Wonder?,spotify:track:5xT8JXJSyHYNfC11OFqzFL,https://open.spotify.com/track/5xT8JXJSyHYNfC1...,['Keane'],Under The Iron Sea,63,3.10,129.352,11,0,0.489,0.955,-2.771,0.0363,0.000038,0.000178,0.3340,0.816,4,piano rock
145,Say You Won't Let Go,spotify:track:5uCax9HTNlzGybIStD3vDh,https://open.spotify.com/track/5uCax9HTNlzGybI...,['James Arthur'],Back from the Edge,88,3.52,85.043,10,1,0.358,0.557,-7.398,0.0590,0.695000,0.000000,0.0902,0.494,4,uk pop
83,You're Gonna Live Forever in Me,spotify:track:51lPx6ZCSalL2kvSrDUyJc,https://open.spotify.com/track/51lPx6ZCSalL2kv...,['John Mayer'],The Search for Everything,74,3.17,136.161,7,1,0.542,0.131,-14.104,0.0316,0.983000,0.005440,0.1560,0.319,4,neo mellow
353,doomsday,spotify:track:4WjxtORnwPavm5PDsAWJEc,https://open.spotify.com/track/4WjxtORnwPavm5P...,['Lizzy McAlpine'],five seconds flat,69,4.47,134.587,1,1,0.404,0.268,-9.631,0.0341,0.583000,0.000000,0.1020,0.124,3,singer-songwriter pop
66,Kill Bill,spotify:track:1Qrg8KqiBpW07V7PNxwwwL,https://open.spotify.com/track/1Qrg8KqiBpW07V7...,['SZA'],SOS,91,2.57,88.980,8,1,0.644,0.735,-5.747,0.0391,0.052100,0.144000,0.1610,0.418,4,pop


In [9]:
new_songs = pd.read_csv("New Playlist.csv")
duration_mins_secs = [str(f"{i}:{j}") for i, j in zip([int(i//1) for i in list(new_songs['Duration'])], [int(round((i%1) * 60, 2)) for i in list(new_songs['Duration'])])]
new_songs['Track Duration'] = duration_mins_secs
new_songs.sample(5)

,Track_Name,Track_ID,URL,Artist(s),Album,Popularity,Duration,Tempo,Key,Mode,Danceability,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveliness,Valence,Time_Signature,Track Duration
175,Merry Christmas,spotify:track:3bH2Eu9A9ymI4UnGHRmoXg,https://open.spotify.com/track/3bH2Eu9A9ymI4Un...,"['Ed Sheeran', 'Elton John']",Merry Christmas,80,3.48,113.990,0,1,0.594,0.935,-2.018,0.0357,0.2430,0.000002,0.1020,0.602,4,3:28
354,21 Guns,spotify:track:64yrDBpcdwEdNY9loyEGbX,https://open.spotify.com/track/64yrDBpcdwEdNY9...,['Green Day'],21st Century Breakdown,77,5.35,159.779,5,1,0.268,0.742,-4.939,0.0355,0.0518,0.000000,0.6260,0.416,4,5:21
130,Precious,spotify:track:3c4bJL7mARZvd387847GsC,https://open.spotify.com/track/3c4bJL7mARZvd38...,"['Depeche Mode', 'Steve Fitzmaurice']",Playing the Angel,63,4.10,104.988,1,1,0.568,0.837,-6.138,0.0266,0.0900,0.058700,0.0994,0.853,4,4:6
188,Casio,spotify:track:44ZKnfWEkp7wPs035j4Tua,https://open.spotify.com/track/44ZKnfWEkp7wPs0...,['Jungle'],For Ever,72,3.91,115.989,10,0,0.816,0.492,-7.464,0.0752,0.4310,0.089600,0.0995,0.765,4,3:54
90,"Hey, Soul Sister",spotify:track:4HlFJV71xXKIGcU3kRyttv,https://open.spotify.com/track/4HlFJV71xXKIGcU...,['Train'],"Save Me, San Francisco (Golden Gate Edition)",85,3.61,97.012,1,0,0.673,0.886,-4.440,0.0431,0.1850,0.000000,0.0826,0.795,4,3:36


In [10]:
cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Acousticness', 'Valence']
scatter_cols = ['Acousticness', 'Danceability', 'Energy', 'Tempo', 'Valence']
hist_cols = ['Popularity', 'Tempo', 'Danceability', 'Energy', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveliness', 'Valence']

---

In [11]:
app1 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app1.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("Comparing Playlists - CDF Plots"),
            dcc.Graph(id="histogram")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app1.callback(
    Output("histogram", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(df, x=field, histfunc='count', nbins=nbins, color='Playlist', color_discrete_sequence=['limegreen', 'fuchsia'], opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        barmode='overlay',
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app1.run(port=8855, debug=True)

---

In [12]:
app2 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app2.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=scatter_cols[0], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=scatter_cols[1], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=scatter_cols[2], options=[{'label': col, 'value': col} for col in scatter_cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app2.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, opacity):
    hover_dict = {xax: False, yax:False, size:False, 'Playlist':False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(df, x=xax, y=yax, size=size, color='Playlist', trendline='ols', color_discrete_sequence=['limegreen', 'fuchsia'], hover_name="Track_Name", hover_data=hover_dict, size_max=25, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app2.run(port=8856, debug=True)

---

In [13]:
app3 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app3.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Radar Plot"),
            dcc.Graph(id="radar"),
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, marks=None, step=None)
            ]),
        ])
    ])
], fluid=True)

@app3.callback(
    Output("radar", "figure"),
    Input('opacity', 'value')
)

def update_figure(opacity):
    trace1 = go.Scatterpolar(
        r = list(p.iloc[0, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'limegreen',
        line = dict(color='black', width=1),
        name='Playlist 1'
    )
    trace2 = go.Scatterpolar(
        r = list(p.iloc[1, :]),
        theta = list(p.columns),
        fill = 'toself',
        fillcolor = 'fuchsia',
        line = dict(color='black', width=1),
        name = 'Playlist 2'
    )
    layout = go.Layout(
        polar = dict(
            radialaxis = dict(
                visible = True,
                range = [0, 1]
            )
        ),
        showlegend = True,
        height = 600,
        width = 1400,
        plot_bgcolor = 'rgba(0, 0, 0, 0)'
    )
    fig = go.Figure(data=[trace1, trace2], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app3.run(port=8857, debug=True)

---

In [14]:
app4 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app4.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Artists"),
            dcc.Graph(id="bubble")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value='Followers', options=[{'label': col, 'value': col} for col in ['Followers', 'Popularity']])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app4.callback(
    Output("bubble", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(measure, topn, opacity):
    hover_dict = {measure: False, 'Index':False, 'Name':False, 'Playlist':False, 'Genres':True, 'Popularity':True, 'Followers':True, 'Total Count':False}
    fig = px.scatter(artists.sort_values(by=measure, ascending=False).head(topn), x='Index', y=measure, text='Name', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'limegreen', 'Playlist 2':'fuchsia', 'Both':'yellow'}, hover_name="Name", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app4.run(port=8858, debug=True)

---

In [15]:
app5 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app5.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Top Genres"),
            dcc.Graph(id="bubble_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app5.callback(
    Output("bubble_too", "figure"),
     [Input('topn', 'value'),
     Input('opacity', 'value')]
)

def update_figure(topn, opacity):
    hover_dict = {'Index':False, 'Genre':False, 'Playlist':False, 'Total Count':True}
    fig = px.scatter(genres.sort_values(by='Total Count', ascending=False).head(topn), x='Total Count', y='Index', text='Genre', size='Total Count', size_max=100, color='Playlist', color_discrete_map={'Playlist 1':'limegreen', 'Playlist 2':'fuchsia', 'Both':'yellow'}, hover_name="Genre", hover_data=hover_dict, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel=dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app5.run(port=8859, debug=True)

---

In [21]:
app6 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app6.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Collaboration Network"),
            dcc.Graph(id="network")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5)
            ])
        ])
    ])
], fluid=True)

@app6.callback(
    Output("network", "figure"),
     Input('opacity', 'value')
)

def update_figure(opacity):

    G = nx.Graph()
    for i, j, k, l, m in zip(list(artists['Name']), list(artists['Genres']), list(artists['Popularity']), list(artists['Followers']), list(artists['Playlist'])):
        G.add_node(i)
        G.nodes[i]['Genres'] = j
        G.nodes[i]['Popularity'] = k
        G.nodes[i]['Followers'] = l
        G.nodes[i]['Playlist'] = m
    for i, j, k, l, m in zip(list(collabs['Artist 1']), list(collabs['Artist 2']), list(collabs['Popular Collab']), list(collabs['Popularity']), list(collabs['Artist From'])):
        G.add_edge(i, j)
        G.edges[(i, j)]['Popular Collab'] = k
        G.edges[(i, j)]['Popularity'] = l
        G.edges[(i, j)]['Artist From'] = m
    nx.write_gexf(G, 'Collaboration Graph.gexf')

    pos = nx.spring_layout(G, k=0.99)
    pos = nx.rescale_layout_dict(pos, 1000)
    for n in G.nodes:
        G.nodes[n]['pos'] = pos[n]

    edge_x = []
    edge_y = []
    xtext = []
    ytext = []
    edgetext = []
    for edge in G.edges():
        x0, y0 = G.nodes[edge[0]]['pos']
        x1, y1 = G.nodes[edge[1]]['pos']
        xtext.append((x0 + x1)/2)
        ytext.append((y0 + y1)/2)
        edgetext.append(f"Artist 1: {edge[0]}, \nArtist 2: {edge[1]},\n Collaboration: {G.edges[edge]['Popular Collab']}")
        edge_x.append(x0)
        edge_x.append(x1)
        edge_x.append(None)
        edge_y.append(y0)
        edge_y.append(y1)
        edge_y.append(None)

    edge_trace = go.Scatter(
        x=edge_x, 
        y=edge_y,
        line = dict(
            width = 1, 
            color = '#888'
        ),
        hoverinfo='text',
        mode='lines'
    )
    edge_trace.text = edgetext

    eweights_trace = go.Scatter(
        x = xtext,
        y = ytext, 
        mode = 'markers',
        marker = dict(
            size = 3,
            color = '#888',
        ),
        text = edgetext,
        hoverinfo = 'text'
    )

    node_x = []
    node_y = []
    for node in G.nodes():
        x, y = G.nodes[node]['pos']
        node_x.append(x)
        node_y.append(y)
    
    node_trace = go.Scatter(
        x = node_x, 
        y = node_y,
        mode = 'markers',
        hoverinfo = 'text',
        marker = dict(
            color = [],
            size = [],
            line_width = 0.5
        )
    )

    pd = {'Playlist 1':'limegreen', 'Playlist 2':'fuchsia', 'Both':'yellow'}

    node_size = []
    node_color = []
    node_text = []
    for n in G.nodes:
        node_size.append((nx.degree_centrality(G)[n] * 500) + 10)
        node_color.append(pd[G.nodes[n]['Playlist']])
        node_text.append(f"Name: {n},\nGenres: {G.nodes[n]['Genres']}")

    node_trace.marker.size = node_size
    node_trace.marker.color = node_color
    node_trace.text = node_text

    layout = go.Layout(
        showlegend = False,
        hovermode = 'closest',
        xaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        yaxis = dict(
            showgrid = False, 
            zeroline = False, 
            showticklabels = False
        ),
        height=1100,
        width=1400,
        plot_bgcolor='rgba(0, 0, 0, 0)'
    )

    fig = go.Figure(data=[edge_trace, eweights_trace, node_trace], layout=layout)
    fig.update_layout(
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity = opacity)
    return fig

app6.run(port=8860, debug=True)

---

In [17]:
app7 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app7.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Genres"),
            dcc.Graph(id="treemap")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app7.callback(
    Output("treemap", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        genre_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Genre', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Purp,
        hover_name = 'Track_Name',
        hover_data = hover_dict,
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        ),
    )
    return fig

app7.run(port=8861, debug=True)

---

In [18]:
app8 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app8.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Wall of Recommendations - Common Artists"),
            dcc.Graph(id="treemap_too")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('Measure'),
                dcc.Dropdown(id='measure', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Top N'),
                dcc.Slider(id='topn', min=5, max=25, marks=None, step=1, value=15)
            ]),
        ])
    ])
], fluid=True)

@app8.callback(
    Output("treemap_too", "figure"), 
    [Input("measure", "value"),
     Input('topn', 'value')]
)

def update_figure(measure, topn):
    hover_dict = {'Artist(s)':True, measure:False, 'Track_Name':False}
    fig = px.treemap(
        artists_songs.sort_values(by=measure, ascending=False).head(topn), 
        path = ['Common Artist', 'Track_Name'],
        values = measure,
        color = measure,
        color_continuous_scale=px.colors.sequential.Purp,
        hover_name = 'Track_Name',
        hover_data = hover_dict
    )
    fig.update_layout(
        width=1400,
        height=600,
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app8.run(port=8862, debug=True)

---

In [19]:
app9 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app9.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='center', children=[
            dbc.Label("Comparing Playlists - Scatterplot"),
            dcc.Graph(id="scatter")
        ]),
        dbc.Col(width=2, align='center', children=[
            html.Div([
                html.Label('X-Axis'),
                dcc.Dropdown(id='xax', value=cols[0], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Y-Axis'),
                dcc.Dropdown(id='yax', value=cols[1], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Size'),
                dcc.Dropdown(id='size', value=cols[2], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Color'),
                dcc.Dropdown(id='color', value=cols[3], options=[{'label': col, 'value': col} for col in cols])
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, value=0.5, step=None, marks=None)
            ]),
        ])
    ])
], fluid=True)

@app9.callback(
    Output("scatter", "figure"), 
    [Input("xax", "value"),
     Input('yax', 'value'),
     Input('size', 'value'),
     Input('color', 'value'),
     Input('opacity', 'value')]
)

def update_figure(xax, yax, size, color, opacity):
    hover_dict = {xax: False, yax:False, size:False, color:False, 'Album':True, 'Artist(s)':True, 'Track Duration':True, 'Popularity':True}
    fig = px.scatter(new_songs, x=xax, y=yax, size=size, color=color, trendline='ols', color_continuous_scale='Purp', hover_name="Track_Name", hover_data=hover_dict, size_max=15, opacity=opacity, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    fig.update_traces(opacity=opacity)
    return fig

app9.run(port=8863, debug=True)

---

In [20]:
app10 = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app10.layout = dbc.Container([
    dbc.Row([
        dbc.Col(width=10, align='left', children=[
            dbc.Label("New Playlist - CDF Plot"),
            dcc.Graph(id="histogram_too")
        ]),
        dbc.Col(width=2, align='right', children=[
            html.Div([
                html.Label('Field'),
                dcc.Dropdown(id='field', value=hist_cols[0], options=[{'label': col, 'value': col} for col in hist_cols])
            ]),
            html.Div([
                html.Label('Bins'),
                dcc.Slider(id='nbins', min=5, max=200, marks=None, step=1, value=10)
            ]),
            html.Div([
                html.Label('Alpha'),
                dcc.Slider(id='opacity', min=0, max=1, marks=None, value=0.5)
            ]),
        ])
    ])
], fluid=True)

@app10.callback(
    Output("histogram_too", "figure"), 
    [Input("field", "value"),
     Input('nbins', 'value'), 
     Input('opacity', 'value')]
)

def update_chart(field, nbins, opacity):
    fig = px.histogram(new_songs, x=field, histfunc='count', nbins=nbins, opacity=opacity, cumulative=True, width=1400, height=600)
    fig.update_layout(
        plot_bgcolor = 'rgba(0, 0, 0, 0)',
        yaxis = {'showgrid': False},
        xaxis = {'showgrid': False},
        hoverlabel = dict(
            bgcolor = "white",
            font_size = 16,
            font_family = "Rockwell"
        )
    )
    return fig

app10.run(port=8864, debug=True)

---